In [70]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input,Dense,Flatten
from keras.layers import LSTM
from numpy import array

In [71]:
df_train = pd.read_csv('./Train_X.csv',encoding= 'cp950')

In [72]:
df_train = df_train.sort_values('日期')
df_test = df_test.sort_values('日期')

stock_list_train = (df_train['股票代號'].unique()).tolist()
stock_list_test = (df_test['股票代號'].unique()).tolist()
data_train = {}
data_test ={}
for name in stock_list_train:
    data_train[name]=df_train[df_train['股票代號']==name].loc[:,['開盤價', '最高價','最低價','收盤價','成交量.股.']]

In [91]:
#定義正規化函式
def normalize(train):
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm

#定義餵入的feature跟label序列
def train_(df, ref_day=10, predict_day=1):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day):
        X_train.append(np.array(df.iloc[i:i+ref_day,:]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day,:]))#['收盤價']))
    return np.array(X_train), np.array(Y_train)

TWA00_train= normalize(data_train['TWA00'])


In [92]:
X_train,Y_train=train_(TWA00_train,7,1)

In [96]:
X_train.shape

ValueError: cannot reshape array of size 8015 into shape (229,5)

In [97]:
a = Y_train.reshape(229,5)

In [89]:
# define model
import keras
units = 50
inputs1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
lstm1=LSTM(units, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True)(inputs1)
lstm2=LSTM(units, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True)(lstm1)
lstm3=LSTM(units, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True)(lstm2)
final=LSTM(5, activation='tanh', recurrent_activation='hard_sigmoid',return_sequences = False)(lstm3)
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.99, beta_2=0.999, amsgrad=False)

model = Model(inputs=inputs1, outputs=final)
model.compile(optimizer=adam, loss='mean_squared_error')


In [98]:
model.fit(X_train, a, epochs = 100, batch_size = 20)

Epoch 1/100
229/229 [==============================] - 4s 20ms/step - loss: 0.0342
Epoch 2/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0158
Epoch 3/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0134
Epoch 4/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0124
Epoch 5/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0134
Epoch 6/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0121
Epoch 7/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0115
Epoch 8/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0105
Epoch 9/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0104
Epoch 10/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0100
Epoch 11/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0091
Epoch 12/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0091
Epoch 13/100

229/229 [==============================] - 0s 1ms/step - loss: 0.0023
Epoch 100/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0022


In [100]:
df_test = pd.read_csv('./Test_X.csv',encoding= 'cp950')


for name in stock_list_train:
    data_test[name]=df_test[df_test['股票代號']==name].loc[:,['開盤價', '最高價','最低價','收盤價','成交量.股.']]
TWA00_test= normalize(data_test['TWA00'])
X_test,Y_test=train_(TWA00_test,7,1)

In [ ]:
predict_y = model.predict(a)

In [ ]:
predict_y

In [ ]:
def denormalize(train):
    denorm = train*(np.max(data_train['TWA00']['收盤價'])-np.min(data_train['TWA00']['收盤價']))+np.mean(data_train['TWA00']['收盤價'])
    return denorm

In [ ]:
Y_test = pd.DataFrame(Y_test)
test_y = denormalize(Y_test)
z = denormalize(predict_y)

In [ ]:
test_y

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(test_y, color = 'red', label = 'Real Price')  # 紅線表示真實股價
plt.plot(z, color = 'blue', label = 'Predicted Price')  # 藍線表示預測股價
plt.title('Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()